<a href="https://colab.research.google.com/github/nilrey/yolo8-cars/blob/main/cars_yolo8_v_bytrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2

In [ ]:
#import torch

In [ ]:

# Загружаем модель YOLOv8
model = YOLO('yolov8x.pt')

In [ ]:
!nvidia-smi

In [ ]:
#model.model.half()

# Принудительно включаем FP16, но безопасно для всех слоев
#for param in model.model.parameters():
#    param.data = param.data.half()

In [ ]:

# Вывод всех классов
#print("Классы, определяемые моделью:")
#for class_id, class_name in model.names.items():
#    print(f"{class_id}: {class_name}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
video_path = '/content/drive/MyDrive/Colab Notebooks/cars/input/s-001.mp4'
cap = cv2.VideoCapture(video_path)
output_path = '/content/drive/MyDrive/Colab Notebooks/cars/output/cars-video-001-s-bytrack-out-08-x.mp4'


In [ ]:

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

allowed_indices = {2, 3, 5, 6, 7, 8}  # Фильтрация классов автомобилей

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))


In [ ]:
print(fps)

25


In [ ]:
frame_skip = 2  # Пропускаем кадры (анализируем 1 из 3)
frame_count = 0  # Счетчик кадров

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Обрабатываем только каждый 3-й кадр
    if frame_count % frame_skip != 0:
        out.write(frame)
        continue

    # Используем AMP для предсказания
    #with torch.autocast(device_type='cuda', dtype=torch.float16):
    #    results = model.track(frame, persist=True, imgsz=640, iou=0.4)

    # Используем модель для анализа текущего кадра с отслеживанием
    results = model.track(frame, persist=True, imgsz=640, iou=0.4) # 0.5

    if results[0].boxes.id is not None:
        for i, box in enumerate(results[0].boxes):
            conf = box.conf[0]
            if int(box.cls[0]) in allowed_indices and conf > 0.5: # 0.7
                xyxy = box.xyxy[0]
                conf = box.conf[0]
                class_name = results[0].names[int(box.cls[0])]
                obj_id = int(results[0].boxes.id[i])  # Получаем ID объекта
                label = f'{class_name} {obj_id} ({conf:.2f})'

                # Рисуем bounding box и ID на кадре
                x1, y1, x2, y2 = map(int, xyxy)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Запись обработанного кадра в выходное видео
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Обработанное видео с трекингом сохранено в {output_path}")



0: 384x640 1 person, 5 cars, 1 truck, 63.2ms
Speed: 2.3ms preprocess, 63.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6 cars, 3 trucks, 63.0ms
Speed: 1.9ms preprocess, 63.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6 cars, 3 trucks, 48.6ms
Speed: 2.0ms preprocess, 48.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6 cars, 3 trucks, 48.6ms
Speed: 2.3ms preprocess, 48.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6 cars, 3 trucks, 42.1ms
Speed: 2.2ms preprocess, 42.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 7 cars, 3 trucks, 41.4ms
Speed: 1.7ms preprocess, 41.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 7 cars, 3 trucks, 40.4ms
Speed: 1.9ms preprocess, 40.4ms inference, 1.7ms postprocess per image at shape (1, 